In [ ]:
import gensim, nltk, os

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

results_dict = dict()
# contains input corpus

# iterate through all files in the corpus folder
for root, dirs, files in os.walk('cleaned_texts_eng'):
    for file_name in files:
        print(file_name)
        with open(os.path.join(root, file_name),encoding='utf8') as rf:
            text = rf.read()
            token_list = pos_tag(word_tokenize(text))
            all_tokens = [lis[0] for lis in token_list]
            all_pos_tags = [lis[1] for lis in token_list]
                
        results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}

In [1]:
import json

#with open("results_dict.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)

with open("results_dict.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [2]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [3]:
stopwords_file = 'stopwords.txt'

with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())

for filename in results_dict:
    noun_token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in noun_token_list if len(word)>=2 and word not in stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)
#     break

In [4]:
texts = []
numbers = []
for item in results_dict.values():
    texts.append(item['cleaned_noun_tokens'])
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)
print(len(numbers))

[202, 812, 1225, 3200, 3324, 3428, 5365, 6933, 7417, 8238, 8940, 9497, 10202, 10447, 10488, 10834, 11234, 11573, 12016, 12142, 13787, 14374, 14414, 14525, 15099, 15605, 15779, 16232, 16394, 16425, 16699, 16778, 17451, 17641, 18150, 18298, 18565, 18595, 18994, 19022, 20834, 21807, 22072, 22791, 23648, 27497, 62527]
47


In [5]:
def split_into_blocks(text, segment_size):
    segment_list = []
    i = 0
    while True:
        left, right = i * segment_size, (i + 1) * segment_size
        segment = text[left:right]
        if not segment:  # If the segment is empty, stop appending
            break
        segment_list.append(segment)
        i += 1
        if right >= len(text):  # This condition is now redundant due to the empty check
            break
    return segment_list


def split_list(l, length=1000):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)

new_texts = []
for text in texts:
     new_texts = new_texts + split_list(text)
print(len(new_texts))
texts = new_texts 

734


In [6]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [7]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [ ]:
word_frequency = dict()
for text in texts:
    for word in text:
        if word not in word_frequency:
            word_frequency[word] = 0
        word_frequency[word] += 1
word_freq_pairs = [(key, val) for key, val in word_frequency.items()]
word_freq_pairs.sort(key=lambda var: var[1], reverse=True)
word_freq_pairs

In [ ]:
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=40,random_state = 1, iterations=1000, chunksize = 1000,eval_every = None)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(5,16)
# z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': y,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()


In [ ]:
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)


In [ ]:
x = range(1,21)
z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
#y = [coherence(i) for i in x]
plt.plot(x, z)
plt.xlabel('主题数目')
plt.zlabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': z,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()

In [8]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics= 6, passes=100,random_state = 1, iterations=300, chunksize = 1000,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.015*"man" + 0.012*"hand" + 0.011*"men" + 0.008*"woman" + 0.008*"face" + 0.007*"heart" + 0.007*"time" + 0.007*"people" + 0.007*"life" + 0.007*"death"'), (1, '0.010*"men" + 0.010*"time" + 0.009*"day" + 0.008*"man" + 0.007*"way" + 0.006*"town" + 0.006*"house" + 0.005*"city" + 0.005*"people" + 0.005*"place"'), (2, '0.015*"time" + 0.010*"way" + 0.010*"water" + 0.009*"day" + 0.007*"mr" + 0.006*"thing" + 0.006*"man" + 0.006*"place" + 0.005*"night" + 0.005*"hand"'), (3, '0.022*"mr" + 0.011*"man" + 0.011*"time" + 0.011*"room" + 0.008*"hand" + 0.008*"way" + 0.007*"doctor" + 0.007*"gentleman" + 0.006*"door" + 0.006*"lady"'), (4, '0.016*"ship" + 0.015*"sea" + 0.011*"time" + 0.011*"boat" + 0.011*"men" + 0.009*"board" + 0.009*"captain" + 0.009*"water" + 0.006*"man" + 0.006*"day"'), (5, '0.009*"country" + 0.008*"time" + 0.006*"day" + 0.006*"court" + 0.005*"people" + 0.005*"person" + 0.005*"master" + 0.005*"part" + 0.004*"world" + 0.004*"year"')]
4
4
4
4
4
4
4
1
1
4
4
4
4
4
4
4
4
4
5
1
2
2
2
2

In [9]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.102243 -0.035417       1        1  36.089038
4      0.045096  0.086438       2        1  19.412116
2      0.080607 -0.016921       3        1  17.782177
1      0.008766  0.014022       4        1  12.394407
3     -0.099756 -0.151247       5        1   8.909512
5     -0.136956  0.103125       6        1   5.412750, topic_info=         Term         Freq        Total Category  logprob  loglift
304        mr  2672.000000  2672.000000  Default  30.0000  30.0000
417      ship  2362.000000  2362.000000  Default  29.0000  29.0000
410       sea  2738.000000  2738.000000  Default  28.0000  28.0000
36       boat  2110.000000  2110.000000  Default  27.0000  27.0000
104   country  1535.000000  1535.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
2489    stone    85.283596   997.894445   Topic6  -6.0697   0.4567
355     place   100.522379  3160.730118   Topic6  -5.9053  -0.5318
410       sea    95.187600  2738.186426   Topic6  -5.9598  -0.4428
258      kind    79.357172   891.563665   Topic6  -6.1417   0.4974
480     thing    81.835022  3161.912325   Topic6  -6.1110  -0.7378

[542 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
17900      6  0.908317   academy
962        1  0.134496   account
962        2  0.300434   account
962        3  0.213098   account
962        4  0.083842   account
...      ...       ...       ...
14926      5  0.038982  yokohama
14926      6  0.896583  yokohama
6066       1  0.997597      zulu
6236       1  0.987053  zululand
6237       1  0.980184     zulus

[1348 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 2, 4, 6])

In [10]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Assuming vis_data is already prepared
# vis_data = gensimvis.prepare(lda_model, corpus, id2word)

# Save to a file
pyLDAvis.save_html(vis, 'lda_result_original.html')


In [ ]:
def plot_token_freq(tokens, topk=100):
    from collections import Counter
    matplotlib.rc("font",family='YouYuan')
    counter = Counter(tokens)
    temp = list(counter.items())
    temp.sort(key=lambda var: var[1], reverse=True)
    temp = temp[:topk]
#     plt.bar([var[0] for var in temp], [var[1] for var in temp])
#     plt.show()
    print(temp)

In [ ]:
for key in results_dict:
    print("小说：", key)
    print("--------")
    plot_token_freq(results_dict[key]['cleaned_noun_tokens'], 100)
    print("================================================================================================================================")